<a href="https://colab.research.google.com/github/QColeman97/AudioTagger/blob/master/AudioTag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns # for data visualization

# import IPython
# import IPython.display as ipd #To play sound in notebook
# import scipy as sci
# import wave
from pathlib import Path

from scipy.fftpack import fft #Fast Fourier Transformation
from scipy.io import wavfile

import librosa
from librosa import display
import os
import glob

input_path = 'drive/My Drive/AudioTaggerData/'


# input_path = '../AudioTaggerData/'
train_files_path = input_path + 'FSDKaggle2018.audio_train'
test_files_path = input_path + 'FSDKaggle2018.audio_test'
train_csv_path = (input_path +
                  'FSDKaggle2018.meta/train_post_competition.csv')
test_csv_path = (input_path +
                 'FSDKaggle2018.meta/' +
                 'test_post_competition_scoring_clips.csv')

#scipy.wavfile.read returns rate of wave, and # of data read
filename = '/001ca53d.wav'
# sample_rate, samples = wavfile.read(str(audio_train_file) + filename)
# print(samples)
# print(train.shape)

# Data preprocessing part

df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

unique_labels = df_train.label.unique()
num_class = len(unique_labels)
label2index = {label: index for index, label in enumerate(unique_labels)}

# print('Label to index:')
# print(label2index)

train_dict = pd.Series.from_csv(train_csv_path, header = 0).to_dict()
test_dict = pd.Series.from_csv(test_csv_path, header = 0).to_dict()


# print('Train dict at:')
# print(test_dict['40f595b9.wav'])
# KeyError: '40f595b9.wav'

# train_df = pd.read_csv(train_csv_path, header = 0)
# train_dict = train_df.transpose().to_dict()
# print('train dict:')
# print(train_dict)

#Indices of manually verified training data
# verified_train = np.array(df_train[df_train.manually_verified == 1].index)
# print(len(verified_train))
# print(len(df_train))

#array of labels in number form (0 = hi-hat, 1 = saxophone, etc)
label_emb_indices = np.array([label2index[label] for label in df_train.label])

label_emb_test_indices = np.array([label2index[label] for label in df_test.label])

# print(plain_y_train)

# print('Label emb indices:')
# print(label_emb_indices)

# # Approach X uses longer sound, then it uses suppressed
# # confX['sampling_rate'] = 26000
# # sampling_rate = 44100 # Original file sr
# sampling_rate = 32000
# # duration = 4
# duration = 5
# # confX['hop_length'] = 520  # 20ms
# hop_length = 192
# fmin = 20
# fmax = sampling_rate // 2
# # confX['n_mels'] = 48
# n_mels = 128
# # confX['n_fft'] = confX['n_mels'] * 20
# n_fft = 1024
# audio_split = 'dont_crop'
# samples = sampling_rate * duration
# dims = (n_mels, 1 + int(np.floor(samples / hop_length)), 1)


def pre_process(pathname):
    sampling_rate = 32000
    # duration = 4
    # duration = 5
    # confX['hop_length'] = 520  # 20ms
    hop_length = 192
    # fmin = 20
    # fmax = sampling_rate // 2
    fmax = None
    # confX['n_mels'] = 48
    n_mels = 128 # originally
#     n_mels = 32
    # confX['n_fft'] = confX['n_mels'] * 20
    n_fft = 1024
    # audio_split = 'dont_crop'
    # samples = sampling_rate * duration
    # dims = (n_mels, 1 + int(np.floor(samples / hop_length)), 1)

    # TODO: Correct spectrogram creation to minimize bad data

    # y, sr = librosa.load(pathname, sr = None)
    y, sr = librosa.load(pathname, sr = sampling_rate)
    # print('Y len:', len(y))
    y, (trim_begin, trim_end) = librosa.effects.trim(y)

    # print('Y len:', len(y))
#     y = librosa.effects.time_stretch(y, 2)
#     print('Y len:', len(y))

    # while len(y) > 10000:
    #     y = librosa.effects.time_stretch(y, 2)
    #     print('Y len:', len(y))
    # print()

    # Amplitudes of STFT
    stft = np.abs(librosa.stft(y, n_fft = n_fft, hop_length = hop_length,
                               window = 'hann', center = True,
                               pad_mode = 'reflect'))

    # print('stft shape:', stft.shape)

    freqs = librosa.core.fft_frequencies(sr = sampling_rate, n_fft = n_fft)
    stft = librosa.perceptual_weighting(stft*2, freqs, ref = 1.0, amin = 1e-10,
                                        top_db = 99.0)

    # print('stft shape:', stft.shape)

    # Apply mel filterbank
    # Power param is set to 2 (power) by default
    mel_spect = librosa.feature.melspectrogram(S = stft, sr = sampling_rate,
                                               n_mels = n_mels, fmax = fmax)

    # print('mel shape:', mel_spect.shape)

    log_mel_spect = librosa.core.power_to_db(mel_spect)

    # print('log mel shape:', log_mel_spect.shape)

    # spectrogram = librosa.feature.melspectrogram(S = stft)
    # Keep spectrogram
    # return np.asarray(spectrogram)
    return np.asarray(log_mel_spect)


# pre_process(audio_train_file + filename)
def get_data(pathname, training = True):
    file_list = glob.glob(os.path.join(pathname, '*.wav'))

    if training:
        data_f = open('Audio.train', 'w')
    else:
        data_f = open('Audio.test', 'w')

    # print(file_list)
#     spectrograms, times = [], []
    spectrograms = np.ndarray((9474, 256, 128))

    for i, file in enumerate(file_list):
        print("%04d / %d | %s" % (i + 1, len(file_list), file))

        try:
            spectrogram = pre_process(file)
        except Exception:
            print('Weird, couldnt convert to spectrogram, skipping file')
            continue

        # times.append(spectrogram.shape[1])

        # Originally 2000 (average)
#         time_restriction = 64
        time_restriction = 256
        if time_restriction >= spectrogram.shape[1]:
            pad_amount = time_restriction - spectrogram.shape[1]
            # Use avg or max time
            spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_amount)),
                                 'minimum')
        else:
            spectrogram = spectrogram[:, :time_restriction]

        spectrogram = spectrogram.transpose()

        # print("Spectrogram Shape:", spectrogram.shape)

#         spectrograms.append(spectrogram.astype(np.float32))
        for j in range(len(spectrogram)):
            for k in range(len(spectrogram[j])):
                spectrograms[i][j][k] = spectrogram[j][k].astype(np.float32)
                
        # data_f.write(np.array2string(spectrogram) + '\n\n')
#         np.savetxt(data_f, spectrogram)
#         data_f.write('\n')

        # if i > 500:
        #     break
        # if 32 < i < 50:  # 34 is a weird one
#         if i % 12 == 0:
        
#             plt.figure("General-Purpose ")
#             plt.clf()
#             plt.subplots_adjust(right = 0.98, left = 0.1, bottom = 0.1,
#                                 top = 0.99)
#             plt.imshow(spectrogram, origin = "lower",
#                        interpolation = "nearest", cmap = "viridis")
#             plt.xlabel("%d bins" % spectrogram.shape[1])
#             plt.ylabel("%d frames" % spectrogram.shape[0])
#             plt.colorbar()
#             plt.show()
        
            # display.specshow(spectrogram, y_axis = 'log', x_axis = 'time')
            #
            # plt.title('Mel Spectrogram')
            # plt.colorbar(format = '%+2.0f dB')
            # plt.tight_layout()
            # plt.show()
        
            # print('Spectrogram:', i)
            # print(spectrogram)

    # average_time = np.average(times)
    # print('Average timesteps:', average_time)
    # max_time = np.amax(times)
    # print('Max timesteps:', max_time)

    return spectrograms
#     data_f.close()


def get_labels(pathname, training = True):
    file_list = glob.glob(os.path.join(pathname, '*.wav'))

    if training:
        labels_f = open('Labels.train', 'w')
    else:
        labels_f = open('Labels.test', 'w')

#     labels = []
    labels = np.ndarray((1570, 41))
    for i, file in enumerate(file_list):
#         label = np.zeros((41,))
        categ = (train_dict[file.split('/')[-1]] if
            (training) else test_dict[file.split('/')[-1]])
        hot_index = label2index[categ]
        labels[i][hot_index] = 1
#         labels.append(label)
#         labels_f.write(' '.join([str(x) for x in label]) + '\n')

    return np.array(labels)
#     labels_f.close()


# get_data(train_files_path, training = True)
# get_labels(train_files_path, training = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4141: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [0]:
from keras import models, layers
from keras_preprocessing.image import ImageDataGenerator
# import librosa
# from librosa import display
import numpy as np
# import matplotlib.pyplot as plt
# import IPython.display as ipd  # only for IPython notebooks
# import pyaudio
# import wave
# import PreProcess


# #1 preprocess method
# Normalize input w/ batch normalization (BN)
# Input is trnasformed into logmel domain (turned into log-mel spectrograms)
# BN again & reshaped (time, frequency, 1) - greyscale image
# Then conv stuff

# Hyperparams / config to test:
# ch 7: (advanced)
#   model ensembling (of 2D CNN & Combined 1D CNN & RNN combo, etc??)
#   Replace Conv2D w/ SeparableConv2D
#   Batch normailization (can replace regularization)
#   Inception
#   Residual connections
#   DenseNet is a good model to follow (top leader) or fine tune from

# ch 5: (convnets)
#   Tune HP (# neurons, layers, epochs, batch_size)
#   dropout, regularization (can be replaced by BN)
#   Data augmentation

# K-Fold Validation


# class LyrParams:
#     # Describes, for a layer, # channels/neurons, dropout rate &
#     # regularization choice (Lyr = layer)
#     def __init__(self, units, dpt = None):
#         self.units = units
#         self.dpt = dpt

# nn.add(layers.Conv2D(prm_lyrs[1].units, (3, 3), activation = 'relu'))
# if prm_lyrs[0].dpt:
#   nn.add(layers.dpt(prm_lyrs[0].dpt))

# Lab 3 is a good one for backup
# Model structure after best methods - 6 conv units (conv *2 + maxpool)
def make_model(input_shape):
    # Current Shape: (500, 128, 1)  # 500 = timesteps, 128 = frequencies
    nn = models.Sequential()
    nn.add(layers.SeparableConv2D(64, (3, 3), padding = 'same',
                                  activation = 'relu',
                                  input_shape = input_shape))
    # Shape: (126, 498, 64)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(64, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (124, 496, 64)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (62, 248, 64)
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv2D(128, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (60, 246, 128)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(128, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (58, 244, 128)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (29, 122, 128)
    nn.add(layers.Dropout(0.3))

#     # Possibly make this block more like the others
#     nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (27, 120, 256)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (25, 118, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (23, 116, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (21, 114, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (10, 57, 256)
    nn.add(layers.Dropout(0.3))

#     nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (8, 55, 512)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (6, 53, 512)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.MaxPooling2D((2, 2)))
#     # Shape: (3, 26, 512)
#     nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (1, 24, 512)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
                                  activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.GlobalAveragePooling2D())

    nn.add(layers.Dense(41, activation = 'softmax'))
    return nn


# Get data
# input_path = 'AudioTaggerData/'
input_path = 'drive/My Drive/AudioTaggerData/'

train_files_path = input_path + 'FSDKaggle2018.audio_train'
test_files_path = input_path + 'FSDKaggle2018.audio_test'
train_csv_path = (input_path +
                  'FSDKaggle2018.meta/train_post_competition.csv')
test_csv_path = (input_path +
                 'FSDKaggle2018.meta/' +
                 'test_post_competition_scoring_clips.csv')


def get_train_data():
#     data = np.ndarray((9474, 256, 128))
    data = np.ndarray((9474, 64, 32))

#     data_len, data = 9474, []
#     data_len, time_len, freq = 9474, 256, 128
    data_len, time_len, freq = 9474, 64, 32


#     data_len, data = 3, []
    with open(input_path + 'Audio.train', 'r') as data_f:
        for i in range(data_len):
#             sample = []
#             time_step = [float(elem) for elem in
#                          data_f.readline().split()]
#             print(time_step)
            for j in range(time_len):
#             while time_step != []:
#                 sample.append(np.array(time_step))
                
    
                time_step = [float(elem) for elem in
                             data_f.readline().split()]
#                 sample.append(time_step)
                
                if len(time_step) < freq:
                    rest = freq - len(time_step)
                    time_step += [-100.0 for x in range(rest)]
    
                for k in range(freq):
#                     if (len(time_step) != 32):
#                         print('BAD time step len:', i)
#                     if (i >= 9474 or j >= 64 or k >= 32):
#                         print('OOB error:')
#                         print(i,j,k)
#                     print(len(time_step))
                    data[i][j][k] = time_step[k]
            
#                 print(time_step)
                # print('line')
                # print(line)
#                 sample.append(np.array(time_step))
#             data.append(np.array(sample))
#             data.append(sample)

            data_f.readline()

#             print('DATA:')
#             print(data)

#     return np.array(data)
    return data

def get_test_data():
#     data = np.ndarray((1570, 256, 128))
    data = np.ndarray((1570, 64, 32))
    data_len = 1570

#     data_len, data = 3, []
    with open(input_path + 'Audio.test', 'r') as data_f:
        for i in range(data_len):

            for j in range(64):
    
                time_step = [float(elem) for elem in
                             data_f.readline().split()]
#                 sample.append(time_step)
                
                if len(time_step) < 32:
                    rest = 32 - len(time_step)
                    time_step += [-100.0 for x in range(rest)]
    
                for k in range(32):
                    data[i][j][k] = time_step[k]

            data_f.readline()

    return data


def get_train_labels():
    labels = np.ndarray((9474, 41))
#     labels_len, labels = 9474, []
    labels_len = 9474

#     labels_len, labels = 3, []
    with open(input_path + 'Labels.train', 'r') as label_f:
        for i in range(labels_len):
            label = [float(elem) for elem in label_f.readline().split()]
#             labels.append(label)

            if len(label) < 41:
                    rest = 41 - len(label)
                    label += [0.0 for x in range(rest)]
            
            for j in range(41):
                labels[i][j] = label[j]

#     return np.array(labels)
    return labels

def get_test_labels():
    labels = np.ndarray((1570, 41))
#     labels_len, labels = 9474, []
    labels_len = 1570

#     labels_len, labels = 3, []
    with open(input_path + 'Labels.test', 'r') as label_f:
        for i in range(labels_len):
            label = [float(elem) for elem in label_f.readline().split()]
#             labels.append(label)

            if len(label) < 41:
                    rest = 41 - len(label)
                    label += [0.0 for x in range(rest)]
            
            for j in range(41):
                labels[i][j] = label[j]

#     return np.array(labels)
    return labels

# 1. Create training data (log mel spectrograms)
# dummy_samples = 10
# Num train samples = 9474
# Num test samples = 1570



train_data = get_train_data()
train_labels = get_train_labels()

test_data = get_test_data()
test_labels = get_test_labels()
# test_data = get_data(test_files_path, training = False)
# test_labels = get_labels(test_files_path, training = False)

train_samples, test_samples = 9474, 1570
max_timesteps = train_data.shape[1]
num_freq = train_data.shape[2]


In [8]:
mean = train_data.mean(axis = 0)
# print('Mean of sample 1:')
# print(mean[0])
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std

# ONCE I HAVE TEST DATA
test_data -= mean
test_data /= std

# Use getter methods here
# dummy_train_data = np.random.random((dummy_samples, dummy_max_timesteps,
#                                      dummy_num_freq, 1))
# # Categorical labels
# dummy_train_labels = np.zeros((10, 41))
# for i, label in enumerate(dummy_train_labels):
#     label[i] = 1
# print(dummy_train_labels)

model = make_model((max_timesteps, num_freq, 1))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    partial_train_data = partial_train_data.reshape(
            (partial_train_data.shape[0], 
             partial_train_data.shape[1], 
             partial_train_data.shape[2], 
             1))
    
    val_data = val_data.reshape(
            (val_data.shape[0], 
             val_data.shape[1], 
             val_data.shape[2], 
             1))
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('2DCNN.h5')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 18s 2ms/step - loss: 3.4965 - acc: 0.0909 - val_loss: 3.7601 - val_acc: 0.0769
Epoch 2/10
7106/7106 [==============================] - 6s 789us/step - loss: 3.1230 - acc: 0.1682 - val_loss: 3.1088 - val_acc: 0.1719
Epoch 3/10
7106/7106 [==============================] - 6s 793us/step - loss: 2.8794 - acc: 0.2174 - val_loss: 3.3704 - val_acc: 0.1482
Epoch 4/10
7106/7106 [==============================] - 6s 813us/step - loss: 2.7567 - acc: 0.2522 - val_loss: 3.0092 - val_acc: 0.2171
Epoch 5/10
7106/7106 [==============================] - 6s 803us/step - loss: 2.6366 - acc: 0.2726 - val_loss: 2.8893 - val_acc: 0.2521
Epoch 6/10
7106/7106 [================

In [9]:


def make_combined_CNN_RNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu',
                                  input_shape = (None, input_shape[-1])))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling1D(3))
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    # nn.add(layers.MaxPooling1D(3))
    # nn.add(layers.Dropout(0.3))
    #
    # nn.add(layers.SeparableConv1D(512, 5, activation = 'relu'))
    # nn.add(layers.BatchNormalization())
    # nn.add(layers.SeparableConv1D(512, 5, activation = 'relu'))
    # nn.add(layers.BatchNormalization())

    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3)))

    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn

model = make_combined_CNN_RNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('1DCNN_RNN.h5')

Train on 7106 samples, validate on 2368 samples
Epoch 1/10


KeyboardInterrupt: ignored

In [14]:
from keras.applications import DenseNet121

def make_dense_net_model(conv_base):
    nn = models.Sequential()
    nn.add(conv_base)
    nn.add(layers.Dense(41, activation = 'softmax'))
    return nn

dn_base = DenseNet121(include_top = False,
                      input_shape = (max_timesteps, num_freq, 3),
                      pooling = 'avg')
dn_base.trainable = True
# print(dn_base.summary())
# Fine-tuning
set_trainable = False
for layer in dn_base.layers:
    if layer.name == 'conv5_block13_0_bn':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = make_dense_net_model(dn_base)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    partial_train_data = partial_train_data.reshape(
            (partial_train_data.shape[0], 
             partial_train_data.shape[1], 
             partial_train_data.shape[2], 
             1))
    
    partial_train_data = np.repeat(partial_train_data, 3, axis=3)
    
    val_data = val_data.reshape(
            (val_data.shape[0], 
             val_data.shape[1], 
             val_data.shape[2], 
             1))
    
    val_data = np.repeat(val_data, 3, axis=3)
#     print('Val Data shape:')
#     print(val_data.shape)

    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('DenseNetModel.h5')

Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 32s 5ms/step - loss: 3.9103 - acc: 0.1036 - val_loss: 11.3045 - val_acc: 0.0439
Epoch 2/10
7106/7106 [==============================] - 2s 327us/step - loss: 2.7763 - acc: 0.2734 - val_loss: 10.1244 - val_acc: 0.0536
Epoch 3/10
7106/7106 [==============================] - 2s 323us/step - loss: 2.2868 - acc: 0.3880 - val_loss: 10.6130 - val_acc: 0.0536
Epoch 4/10
6144/7106 [========================>.....] - ETA: 0s - loss: 1.8867 - acc: 0.5085

KeyboardInterrupt: ignored

In [12]:


def make_1DCNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu',
                                  input_shape = (None, input_shape[-1])))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling1D(5))
    nn.add(layers.Dropout(0.3))

#     nn.add(layers.SeparableConv1D(128, 7, activation = 'relu'))
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.SeparableConv1D(128, 7, activation = 'relu'))
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.MaxPooling1D(5))
#     nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.GlobalAveragePooling1D())

    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn

model = make_1DCNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('1DCNN.h5')


Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 6s 847us/step - loss: 3.6751 - acc: 0.0712 - val_loss: 3.4487 - val_acc: 0.1174
Epoch 2/10
7106/7106 [==============================] - 0s 51us/step - loss: 3.3391 - acc: 0.1337 - val_loss: 3.3128 - val_acc: 0.1360
Epoch 3/10
7106/7106 [==============================] - 0s 48us/step - loss: 3.1926 - acc: 0.1632 - val_loss: 3.2315 - val_acc: 0.1639
Epoch 4/10
7106/7106 [==============================] - 0s 50us/step - loss: 3.0928 - acc: 0.1765 - val_loss: 3.1615 - val_acc: 0.1723
Epoch 5/10
7106/7106 [==============================] - 0s 48us/step - loss: 3.0095 - acc: 0.1931 - val_loss: 3.1026 - val_acc: 0.1883
Epoch 6/10
7106/7106 [==============================] - 0s 48us/step - loss: 2.9357 - acc: 0.2100 - val_loss: 3.0552 - val_acc: 0.1862
Epoch 7/10
7106/7106 [==============================] - 0s 51us/step - loss: 2.8864 - acc: 0.2204 - val_loss: 3.0285 - val_acc: 0.1938
Epoch 

In [13]:
def make_RNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True,
                                            input_shape =
                                            (None, input_shape[-1]))))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3)))
    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn


model = make_RNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('RNN.h5')


Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 19s 3ms/step - loss: 3.6112 - acc: 0.0671 - val_loss: 3.4713 - val_acc: 0.0887
Epoch 2/10
7106/7106 [==============================] - 8s 1ms/step - loss: 3.4019 - acc: 0.1088 - val_loss: 3.3418 - val_acc: 0.1204
Epoch 3/10
7106/7106 [==============================] - 8s 1ms/step - loss: 3.2729 - acc: 0.1337 - val_loss: 3.2336 - val_acc: 0.1584
Epoch 4/10
7106/7106 [==============================] - 8s 1ms/step - loss: 3.1795 - acc: 0.1621 - val_loss: 3.1622 - val_acc: 0.1736
Epoch 5/10
7106/7106 [==============================] - 8s 1ms/step - loss: 3.1167 - acc: 0.1717 - val_loss: 3.1093 - val_acc: 0.1795
Epoch 6/10
7106/7106 [==============================] - 9s 1ms/step - loss: 3.0486 - acc: 0.1860 - val_loss: 3.0596 - val_acc: 0.1888
Epoch 7/10
7106/7106 [==============================] - 9s 1ms/step - loss: 3.0071 - acc: 0.1939 - val_loss: 3.0253 - val_acc: 0.2052
Epoch 8/10
71

In [0]:
# Reorganize all data as sets of flat vectors of 0-1.0 values
# Shape (..., 28, 28, 1)

test_data_2D = test_data.reshape(
               (test_data.shape[0], 
                test_data.shape[1], 
                test_data.shape[2], 
                1))
    
test_data_2D_3chan = np.repeat(test_data_2D, 3, axis=3)
    

cnn2D = models.load_model('2DCNN.h5')
rnn = models.load_model('RNN.h5')
cnn1D = models.load_model('1DCNN.h5')
cnn_rnn = models.load_model('1DCNN_RNN.h5')
dense_net = models.load_model('DenseNetModel.h5')

In [19]:
cnn2D_preds = cnn2D.predict(test_data_2D)
rnn_preds = rnn.predict(test_data)
cnn1D_preds = cnn1D.predict(test_data)
cnn_rnn_preds = cnn_rnn.predict(test_data)
dense_net_preds = dense_net.predict(test_data_2D_3chan)

final_preds = 0.2 * (cnn2D_preds + rnn_preds + cnn1D_preds + 
                     cnn_rnn_preds + dense_net_preds)

KeyboardInterrupt: ignored

In [27]:
print(final_preds)

with open('TestTags.out', 'w') as ttf:
    test_file_list = glob.glob(os.path.join(test_files_path, '*.wav'))
    for i, filename in enumerate(test_file_list[:1570]):
#     for i, res in enumerate(final_preds):
        ttf.write(str(i) + ') ' + filename + ' - ' + 
                  unique_labels[np.argmax(final_preds[i])] + '\n')
#                   unique_labels[final_preds[i].tolist().index(1.)] + '\n')

[[1.8031243e-02 9.5451373e-04 2.8195424e-02 ... 2.4594639e-03
  2.0291019e-02 2.7335644e-02]
 [1.8031243e-02 9.5451373e-04 2.8195424e-02 ... 2.4594639e-03
  2.0291019e-02 2.7335644e-02]
 [6.2514038e-05 9.0502948e-03 6.6893133e-03 ... 7.8340154e-04
  2.1446283e-01 6.3963613e-05]
 ...
 [1.2265375e-02 1.5179233e-03 1.5662372e-02 ... 2.8284453e-03
  2.4859700e-02 1.9797010e-02]
 [3.1991460e-04 4.6410818e-02 3.8418453e-02 ... 2.9441938e-03
  9.0320796e-02 4.0703779e-04]
 [9.5287582e-04 3.1554386e-02 2.0984648e-02 ... 2.3450930e-02
  1.2461608e-01 4.0440631e-04]]


In [28]:
cnn2D_loss = cnn2D.evaluate(test_data_2D, test_labels)
rnn_loss = rnn.evaluate(test_data, test_labels)
cnn1D_loss = cnn1D.evaluate(test_data, test_labels)
cnn_rnn_loss = cnn_rnn.evaluate(test_data, test_labels)
dense_net_loss = dense_net.evaluate(test_data_2D_3chan, test_labels)

print('2DCNN Test Loss:', cnn2D_loss)
print('1DCNN Test Loss:', cnn1D_loss)
print('Combined 1D CNN & RNN Test Loss:', cnn_rnn_loss)
print('Dense Net Conv. Base + Classifier Test Loss:', dense_net_loss)
print('RNN Test Loss:', rnn_loss)

KeyboardInterrupt: ignored